In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow import keras

# Denkfehler!

Accuracy is not bad though.

In [2]:
train = pd.read_csv('trainSet.txt')
test = pd.read_csv('testSet.txt')

## Autoencoder should not work :D 

In [3]:
def create_model():
    
    regularizer = keras.regularizers.l1_l2(l1=0.0, l2=0.5)
    
    inputs = keras.Input(shape=(16,))
    
    x = keras.layers.Dense(16, activation='relu')(inputs)
    # build a little auto-encoder
    x = keras.layers.Dense(250, activation='relu', kernel_regularizer=regularizer)(x)
    x = keras.layers.Dropout(0.25)(x)

    
    out1 = keras.layers.Dense(16, activation='sigmoid')(x)
    out2 = keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs, [out1, out2])
    
    return model

In [4]:
# columns to remove:
cols_out = ['PatientID', 'ImageFile', 'Hospital', 'Prognosis']

train_clean = train[[i for i in train.columns if i not in cols_out]]
train_y = train['Prognosis'] == 'MILD'

In [5]:
# Factorize hospitals, and use hospitals for cross-validation
train.Hospital, mapping_index = train.Hospital.factorize()
PF = PredefinedSplit(train['Hospital'])

In [6]:
early_stopping = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [9]:
accs, mses, mses2 = [], [], []

train_mask = np.isnan(train_clean).values

for n, (tr, vl) in enumerate(PF.split(train)):
    model = create_model()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
                  loss=['mse', 'binary_crossentropy'])
    MM = MinMaxScaler()
    
    # Remove Nans
    x_train = train_clean.iloc[tr].copy().fillna(0)
    x_valid = train_clean.iloc[vl].copy().fillna(0)
    # Define y-values
    y_train = train_y.iloc[tr]
    y_valid = train_y.iloc[vl]
    # Scale data from 0 to 1
    x_train = MM.fit_transform(x_train)
    x_valid = MM.transform(x_valid)
    
    # Set former nan values to -1
    #x_train[train_mask[tr]] = -1
    #x_valid[train_mask[vl]] = -1
    
    # Fit Model
    model.fit(x_train, [x_train, y_train], verbose=0, callbacks=[early_stopping], validation_split=0.15,
             epochs=250)
    
    # Predict the things
    pred_val, pred_y = model.predict(x_valid)
    
    _, train_mse, train_acc = model.evaluate(x_train, [x_train, y_train], verbose=0)
    
    mses.append(mean_squared_error(MM.inverse_transform(x_valid), MM.inverse_transform(pred_val)))
    mses2.append(mean_squared_error(x_valid, pred_val))
    accs.append(accuracy_score(y_valid, pred_y > 0.5))
    
    print(f"Split {mapping_index[n]}: MSE - {mses[-1]:4.2f}" +
          f" Acc - {accs[-1]:4.2f}" +
          f" MSES2 - {mses2[-1]:4.2f}")
    
    print(f"Train - MSE {train_mse:4.3f}, ACC {train_acc:4.3f}")

Split D: MSE - 2046692.40 Acc - 0.72 MSES2 - 0.11
Train - MSE 0.096, ACC 0.624
Split E: MSE - 125171.19 Acc - 0.64 MSES2 - 0.07
Train - MSE 0.097, ACC 0.596
Split F: MSE - 838455.56 Acc - 0.64 MSES2 - 0.11
Train - MSE 0.094, ACC 0.523
Split B: MSE - 777479.13 Acc - 0.62 MSES2 - 0.12
Train - MSE 0.092, ACC 0.592
Split C: MSE - 228335.31 Acc - 0.52 MSES2 - 0.14
Train - MSE 0.095, ACC 0.612
